In [1]:
import urllib3, shutil
import requests
import os
from bs4 import BeautifulSoup
from datetime import date,datetime
def clearFiles(path, debug = False):
    for filename in os.listdir(path):
        if (filename != ".DS_Store"): os.remove(path + filename)
url1 = 'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?'
url = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID=1&year1=2019&year2=2020&type=Mean"
year1 = '1981'
year2 = '2020'
def getPath(id, path):
    today = date.today()
    now = datetime.now()
    date_and_time_time = now.strftime("%m.%d.%Y.%H;%M;%S")
    #return "data/"+str(id)+"z_"+date_and_time_time+namespaceOld[id]+".txt"
    return path+str(id)+"z_"+date_and_time_time+".csv"
def getData(id, path, debug = False):
    urlLocal = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID="+str(id)+"&year1=2019&year2=2020&type=Mean"
    URL = url1+'country='+'Ukraine'+'&provinceID='+str(id)+'&year1='+year1+'&year2='+year2+'&type=Mean'
    http = urllib3.PoolManager()
    path = getPath(id, path)
    with http.request('GET', URL, preload_content=False) as res, open(path, 'wb') as out_file:       
        shutil.copyfileobj(res, out_file)
    with open(path, 'r') as file:
        data = file.readlines()
        #print(data)
        if(debug): print(data[0].rfind("<pre>") + " — this is useless information for us right now")
        line = data[0]
        line = line.split("<pre>")[1]
        line = line + line #Скопіюємо першу строку щоб данні не загубилися у датафреймі
        #print(line)
        data[0] = line
        #print(data[0])
        line = ''
        data[len(data) - 1] = line
        with open(path, 'w') as file:
            file.writelines(data)
            file.close
clearFiles("data/")
i = 1;
while i<=27:
    getData(i, "data/")
    i=i+1

In [2]:
idDictionary = {'1':'22', '2':'24', '3':'23', '4':'25', 
    '5':'3', '6':'4', '7':'8', '8':'21', 
    '9':'22', '10':'21', '11':'9', '12':'26', #26 — місто Київ
    '13':'10', '14':'11', '15':'12', '16':'13', 
    '17':'14', '18':'15', '19':'16', '20':'27', #27 — місто Севастополь
    '21':'17', '22':'18', '23':'6', '24':'1', 
    '25':'2', '26':'7' , '27':'5'}
def changeIDs(path, debug = False):
    if(debug): print(idDictionary)
    for filename in os.listdir(path): 
        filePath = path+ filename 
        if(debug): print(filename)
        splitName = filename.split("z_")
        if len(splitName) == 1:
            if(debug): print('skipping name change')
            continue
        if(debug): 
            print(idDictionary)
        newFileName = idDictionary[splitName[0]] + '_'+ splitName[1]
        os.rename(filePath, path + newFileName)
        if (debug):
            print("renamed")
            print(newFileName)
changeIDs('data/')

In [3]:
import pandas as pd
def formFrameByID(ID, path, debug = False):
    ID = str(ID)
    listOfFiles=os.listdir(path)
    #print(listOfFiles)
    for i in listOfFiles:
        idRead = i.split('_')[0]
        if (debug): print(idRead)
        if (ID == idRead):
            df = pd.read_csv(path+i)
            if (debug): print(path+i)
            columns_names=df.columns.values
            df = df.rename(columns=
                {columns_names[0]: 'Year',columns_names[1]:'Week', columns_names[2] :'NDVI' , columns_names[3] : 'BT',
                columns_names[4] : 'VC',columns_names[5] : 'TC', columns_names[6] : 'VHI'})
            df= df.drop('Unnamed: 7', 1)
            if(type(df)!= 'NoneType'): return df
df = formFrameByID(1, 'data/')
print(df)

      Year  Week  NDVI   BT   VC   TC  VHI
0     1981     1  -1.0 -1.0 -1.0 -1.0 -1.0
1     1981     2  -1.0 -1.0 -1.0 -1.0 -1.0
2     1981     3  -1.0 -1.0 -1.0 -1.0 -1.0
3     1981     4  -1.0 -1.0 -1.0 -1.0 -1.0
4     1981     5  -1.0 -1.0 -1.0 -1.0 -1.0
...    ...   ...   ...  ...  ...  ...  ...
2075  2020    48  -1.0 -1.0 -1.0 -1.0 -1.0
2076  2020    49  -1.0 -1.0 -1.0 -1.0 -1.0
2077  2020    50  -1.0 -1.0 -1.0 -1.0 -1.0
2078  2020    51  -1.0 -1.0 -1.0 -1.0 -1.0
2079  2020    52  -1.0 -1.0 -1.0 -1.0 -1.0

[2080 rows x 7 columns]


In [4]:
def sampleSelection(ID, path, debug = False):
    df = formFrameByID(ID, path)
    minVhiArr, maxVhiArr = [],[]
    for i in range(1982, 2020):
        min_v = df[(df.Year.astype(str)==str(i)) & df.VHI>0]['VHI'].min()
        max_v = df[(df.Year.astype(str)==str(i)) & df.VHI>0]['VHI'].max()
        minVhiArr.append(min_v)
        maxVhiArr.append(max_v)
    df_extremum = pd.DataFrame({"Year":[i for i in range(1982, 2020)], 'Min' : minVhiArr, 'Max': maxVhiArr})
    df_drought = df[(df.VHI<=15) & (df.VHI>0)]
    return df_extremum, df_drought
df_ext1, df_dr1 = sampleSelection(26, "data/")
print(df_ext1)
print(df_dr1)

    Year    Min    Max
0   1982  40.98  54.14
1   1983  31.63  65.51
2   1984  -1.00  69.17
3   1985  -1.00  38.38
4   1986  17.09  38.97
5   1987  30.04  57.91
6   1988  32.62  62.85
7   1989  27.86  51.46
8   1990  33.32  56.93
9   1991  38.59  63.83
10  1992  42.45  86.57
11  1993  44.72  75.52
12  1994  -1.00  81.37
13  1995  -1.00  61.63
14  1996  13.14  54.32
15  1997  15.93  46.83
16  1998  34.02  75.70
17  1999  36.67  68.39
18  2000  21.08  70.50
19  2001   9.73  57.98
20  2002  25.35  56.44
21  2003  39.43  78.49
22  2004  -1.00  56.62
23  2005  30.09  60.47
24  2006  28.14  50.05
25  2007  27.02  51.65
26  2008  11.96  47.91
27  2009  32.21  69.43
28  2010  18.91  52.86
29  2011  13.42  45.89
30  2012  30.23  51.88
31  2013  26.20  49.13
32  2014  25.55  53.10
33  2015  32.91  64.48
34  2016  25.65  56.93
35  2017  29.46  49.45
36  2018  14.15  41.36
37  2019  32.73  55.15
      Year  Week   NDVI      BT     VC     TC    VHI
830   1996    51  0.051  287.06  23.97   4.23  14.

In [5]:
idToNames = {
    1:'Вінницька',2:'Волинська',3:'Дніпропетровська',4:'Донецька',
    5:'Житомирська',6:'Закарпатська',7:'Запорізька',8:'Івано-Франківська',
    9:'Київська',10:'Кіровоградська',11:'Луганська',12:'Львівська',
    13:'Миколаївська',14:'Одеська',15:'Полтавська',16:'Рівенська',
    17:'Сумська',18:'Тернопільська',19:'Харківська',20:'Херсонська',
    21:'Хмельницька',22:'Черкаська',23:'Чернівецька',24:'Чернігівська',
    25:'Республіка Крим',26:'Київ',27:'Севастополь'
}
def createLargeDF(path = "data/", debug = False):
    dataframeSum = formFrameByID(1, path, debug)
    dataframeSum.insert(7,'id', 1)
    dataframeSum.insert(8,'name', idToNames[1])
    for i in range (2, len(os.listdir(path))): 
        df = formFrameByID(i, path, debug)
        if(df is not None):
            df.insert(7,'id', i) 
            df.insert(8,'name', idToNames[i])
            if(debug): print(df)
            dataframeSum = dataframeSum.append(df)
    return dataframeSum
print(createLargeDF("data/", False))

      Year  Week  NDVI   BT   VC   TC  VHI  id       name
0     1981     1  -1.0 -1.0 -1.0 -1.0 -1.0   1  Вінницька
1     1981     2  -1.0 -1.0 -1.0 -1.0 -1.0   1  Вінницька
2     1981     3  -1.0 -1.0 -1.0 -1.0 -1.0   1  Вінницька
3     1981     4  -1.0 -1.0 -1.0 -1.0 -1.0   1  Вінницька
4     1981     5  -1.0 -1.0 -1.0 -1.0 -1.0   1  Вінницька
...    ...   ...   ...  ...  ...  ...  ...  ..        ...
2075  2020    48  -1.0 -1.0 -1.0 -1.0 -1.0  26       Київ
2076  2020    49  -1.0 -1.0 -1.0 -1.0 -1.0  26       Київ
2077  2020    50  -1.0 -1.0 -1.0 -1.0 -1.0  26       Київ
2078  2020    51  -1.0 -1.0 -1.0 -1.0 -1.0  26       Київ
2079  2020    52  -1.0 -1.0 -1.0 -1.0 -1.0  26       Київ

[49920 rows x 9 columns]
